### Files

For all files the values were aggregated over 6 models (NN, LGR, RF, DT, SVM, NB) with 40 iterations each. 

- CHANGE_(attr).csv -> change percentage from the base model
- MEAN_(attr).csv -> mean values over all runs
- STD_(attr).csv -> std of values from all runs

### RQ1: WHICH LOSS (initial thoughts)

Rule out based on prediction performance on adult (compas has good performance for all):
- FL the lowest performance on all 4 metrics
- FK, L, VAE, also significantly lowest metrics for adult sex and very high variance
- F 2nd lowest performance and high var on adult race

- FP, LP, P, KL, KP, K all have similar results 

note that all well performing losses seem to use either K or P


BIAS mit performance?

KL, KP, K seem to have consistently good DI, SF, ASPD, for botgh datasets and attributes

PLAN: 
keep going with the KL, KP, K
KL - is advrserial so ideally would rule it out due to time constraints. if K does not reliably perform this will be suggested as the alternative without using Y labels!!!
KP - seems best BUT uses Y. (justify KP by the lower var on adult race aod???? ... sus)
K - seems bit more variance and less debias but will wait for enough evidence

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap

from experiment_utils import *
from src import *


pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'


# SAVE TO RESULTS FILE

In [2]:
FOLDER = "RQ1_LOSS_SELECTION"

if not os.path.exists(FOLDER):
    os.makedirs(FOLDER)

In [3]:
# for each file and each dataset get
#   mean value for all metrics
#   mean change for all metrics
#   mean std for all metrics
# averaged over all 6 models

In [4]:
for attr in [["sex"],["race"],["race", "sex"]]:            
    files = [os.path.join("results","RESULTS_"+'_'.join(attr)+".csv")]
    
    reader = ResultsReader(files)
    
    df = reader.get_relative_metrics(use_percent=True)
    df[reader.ML] = ''
    mean_df = df.groupby(reader.filterable).mean().reset_index()
    mean_df.to_csv( os.path.join(FOLDER,"CHANGE_"+'_'.join(attr)+'.csv'), index=False)
    
    df = reader.get_mean_metrics()
    df[reader.ML] = ''
    mean_df = df.groupby(reader.filterable).mean().reset_index()
    mean_df.to_csv(os.path.join(FOLDER,"MEAN_"+'_'.join(attr)+'.csv'), index=False)
    
    df = reader.get_filtered_df()
    df[reader.ML] = ''
    mean_df = df.groupby(reader.filterable).std().reset_index()
    mean_df.to_csv(os.path.join(FOLDER,"STD_"+'_'.join(attr)+'.csv'), index=False)



File 'results\RESULTS_sex.csv' successfully loaded as DataFrame.
File 'results\RESULTS_race.csv' successfully loaded as DataFrame.
File 'results\RESULTS_race_sex.csv' successfully loaded as DataFrame.
